In [1]:
from langchain_groq import ChatGroq
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import Chroma, FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import os
load_dotenv()
groq_key = os.getenv("GROQ")
groq_key

c:\Users\Public\anaconda3\envs\langchaingroq\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'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'

In [2]:
groq_model = ChatGroq(
    model="llama3-70b-8192",
    groq_api_key=groq_key)

In [3]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
loader = WebBaseLoader(
    web_paths=(
        "https://lilianweng.github.io/posts/2023-06-23-agent/",
    ),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=(
                "post-content", "post-title", "post-header"
            )
        )
    )
)
docs = loader.load()

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
)

In [10]:
splitted_docs = text_splitter.split_documents(docs)

In [11]:
vectore_store = Chroma.from_documents(
    documents=splitted_docs, embedding=HuggingFaceEmbeddings()
)

C:\Users\Bichu George\AppData\Local\Temp\ipykernel_17736\3198811879.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  documents=splitted_docs, embedding=HuggingFaceEmbeddings()
C:\Users\Bichu George\AppData\Local\Temp\ipykernel_17736\3198811879.py:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  documents=splitted_docs, embedding=HuggingFaceEmbeddings()


In [12]:
retriver = vectore_store.as_retriever()

In [13]:
prompt = hub.pull(
    "rlm/rag-prompt"
)

c:\Users\Public\anaconda3\envs\langchaingroq\lib\site-packages\langsmith\client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [18]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [19]:
rag_chain = (
    {
        "context": retriver | format_docs, 
        "question": RunnablePassthrough()
    }
    | prompt
    | groq_model
    | StrOutputParser()
)

In [20]:
rag_chain.invoke(
    "What is Task Composition?"
)

'Task composition refers to the process of breaking down a complex task into smaller, manageable subtasks that can be executed individually. This decomposition enables the identification of multiple reasoning possibilities at each step, creating a tree structure that can be searched using techniques like breadth-first search (BFS) or depth-first search (DFS).'